In [1]:
## Here I am dividing the data first based onto protected attribute value and then train two separate models

import pandas as pd
import random,time,csv
import numpy as np
import math,copy,os
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import sklearn.metrics as metrics

import sys
sys.path.append(os.path.abspath('..'))

from Measure import measure_final_score,calculate_recall,calculate_F1,calculate_precision,calculate_accuracy

In [2]:
## Load dataset
from sklearn import preprocessing
dataset_orig = pd.read_csv('../dataset/adult.data.csv')

## Drop NULL values
dataset_orig = dataset_orig.dropna()

## Drop categorical features
dataset_orig = dataset_orig.drop(['workclass','fnlwgt','education','marital-status','occupation','relationship','native-country'],axis=1)

## Change symbolics to numerics
dataset_orig['sex'] = np.where(dataset_orig['sex'] == ' Male', 1, 0)
dataset_orig['race'] = np.where(dataset_orig['race'] != ' White', 0, 1)
dataset_orig['Probability'] = np.where(dataset_orig['Probability'] == ' <=50K', 0, 1)


## Discretize age
dataset_orig['age'] = np.where(dataset_orig['age'] >= 70, 70, dataset_orig['age'])
dataset_orig['age'] = np.where((dataset_orig['age'] >= 60 ) & (dataset_orig['age'] < 70), 60, dataset_orig['age'])
dataset_orig['age'] = np.where((dataset_orig['age'] >= 50 ) & (dataset_orig['age'] < 60), 50, dataset_orig['age'])
dataset_orig['age'] = np.where((dataset_orig['age'] >= 40 ) & (dataset_orig['age'] < 50), 40, dataset_orig['age'])
dataset_orig['age'] = np.where((dataset_orig['age'] >= 30 ) & (dataset_orig['age'] < 40), 30, dataset_orig['age'])
dataset_orig['age'] = np.where((dataset_orig['age'] >= 20 ) & (dataset_orig['age'] < 30), 20, dataset_orig['age'])
dataset_orig['age'] = np.where((dataset_orig['age'] >= 10 ) & (dataset_orig['age'] < 10), 10, dataset_orig['age'])
dataset_orig['age'] = np.where(dataset_orig['age'] < 10, 0, dataset_orig['age'])



from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dataset_orig = pd.DataFrame(scaler.fit_transform(dataset_orig),columns = dataset_orig.columns)



# divide the data based on sex
dataset_orig_male , dataset_orig_female = [x for _, x in dataset_orig.groupby(dataset_orig['race'] == 0)]

print(dataset_orig_male.shape)
print(dataset_orig_female.shape)

(41762, 8)
(7080, 8)


c:\users\jchakra\python\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int32, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


# Check Original Scores

In [3]:
print(dataset_orig.shape)
# np.random.seed(0)
## Divide into train,validation,test
dataset_orig_train, dataset_orig_test = train_test_split(dataset_orig, test_size=0.2, random_state=0,shuffle = True)

X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'Probability'], dataset_orig_train['Probability']
X_test , y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']

# --- LSR
clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100)


print("recall :", measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'recall'))
print("far :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'far'))
print("precision :", measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'precision'))
print("accuracy :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'accuracy'))
print("F1 Score :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'F1'))
print("aod sex:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'aod'))
print("eod sex:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'eod'))

print("TPR:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'TPR'))
print("FPR:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'FPR'))

(48842, 8)
7090 382 1333 964
899 5923 1187 368 65 1167 146 14
recall : 0.42
7090 382 1333 964
899 5923 1187 368 65 1167 146 14
far : 0.05
7090 382 1333 964
899 5923 1187 368 65 1167 146 14
precision : 0.72
7090 382 1333 964
899 5923 1187 368 65 1167 146 14
accuracy : 0.82
7090 382 1333 964
899 5923 1187 368 65 1167 146 14
F1 Score : 0.53
7090 382 1333 964
899 5923 1187 368 65 1167 146 14
FPR_male: 0.0584962645048482 FPR_female: 0.011854360711261643
TPR_male: 0.43096836049856185 TPR_female: 0.3080568720379147
aod sex: 0.03
7090 382 1333 964
899 5923 1187 368 65 1167 146 14
TPR_male: 0.43096836049856185 TPR_female: 0.3080568720379147
eod sex: 0.12
7090 382 1333 964
899 5923 1187 368 65 1167 146 14
TPR_male: 0.43096836049856185 TPR_female: 0.3080568720379147
TPR: 0.12
7090 382 1333 964
899 5923 1187 368 65 1167 146 14
FPR_male: 0.0584962645048482 FPR_female: 0.011854360711261643
FPR: -0.05


# Train the model for male

In [4]:
dataset_orig_male['race'] = 0
X_train_male, y_train_male = dataset_orig_male.loc[:, dataset_orig_male.columns != 'Probability'], dataset_orig_male['Probability']
# --- LSR
clf_male = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100)
clf_male.fit(X_train_male, y_train_male)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

# Train the model for female

In [5]:
X_train_female, y_train_female = dataset_orig_female.loc[:, dataset_orig_female.columns != 'Probability'], dataset_orig_female['Probability']
# --- LSR
clf_female = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100)
clf_female.fit(X_train_female, y_train_female)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

# Remove biased rows

In [6]:
print(dataset_orig.shape)

df_removed = pd.DataFrame(columns=dataset_orig.columns)

for index,row in dataset_orig.iterrows():
    row_ = [row.values[0:len(row.values)-1]]
    y_male = clf_male.predict(row_)
    y_female = clf_female.predict(row_)
    if y_male[0] != y_female[0]:        
        df_removed = df_removed.append(row, ignore_index=True)
        dataset_orig = dataset_orig.drop(index)
        
print(dataset_orig.shape)
print(df_removed.shape)


(48842, 8)
(46527, 8)
(2315, 8)


# Store unbiased data into new csv 

In [7]:
# fp = open('Adult_Sex.csv', 'w' , newline='')
# myFile = csv.writer(fp)
# column_names = ['age', 'education-num', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'Probability']
# myFile.writerow(column_names)


# for index,row in dataset_orig.iterrows():
#     myFile.writerow(row)

# Train new model on unbiased data

In [8]:
print(dataset_orig.shape)
# np.random.seed(0)
## Divide into train,validation,test
dataset_orig_train, dataset_orig_test = train_test_split(dataset_orig, test_size=0.2, random_state=0,shuffle = True)

X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'Probability'], dataset_orig_train['Probability']
X_test , y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']

# --- LSR
clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100)


print("recall :", measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'recall'))
print("far :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'far'))
print("precision :", measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'precision'))
print("accuracy :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'accuracy'))
print("F1 Score :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'F1'))
print("aod sex:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'aod'))
print("eod sex:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'eod'))

print("TPR:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'TPR'))
print("FPR:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'FPR'))



(46527, 8)
7009 250 1345 702
647 5826 1226 233 55 1183 119 17
recall : 0.34
7009 250 1345 702
647 5826 1226 233 55 1183 119 17
far : 0.03
7009 250 1345 702
647 5826 1226 233 55 1183 119 17
precision : 0.74
7009 250 1345 702
647 5826 1226 233 55 1183 119 17
accuracy : 0.83
7009 250 1345 702
647 5826 1226 233 55 1183 119 17
F1 Score : 0.47
7009 250 1345 702
647 5826 1226 233 55 1183 119 17
FPR_male: 0.038455190625515764 FPR_female: 0.014166666666666666
TPR_male: 0.34543513080619326 TPR_female: 0.3160919540229885
aod sex: 0.0
7009 250 1345 702
647 5826 1226 233 55 1183 119 17
TPR_male: 0.34543513080619326 TPR_female: 0.3160919540229885
eod sex: 0.03
7009 250 1345 702
647 5826 1226 233 55 1183 119 17
TPR_male: 0.34543513080619326 TPR_female: 0.3160919540229885
TPR: 0.03
7009 250 1345 702
647 5826 1226 233 55 1183 119 17
FPR_male: 0.038455190625515764 FPR_female: 0.014166666666666666
FPR: -0.02
